In [1]:
import pandas as pd
df = pd.read_csv('..\data.csv')
df.head()
nans = df.loc[pd.isna(df["isSpam"]), :].index
for n in nans:
    print(df.iloc[n])

In [2]:
df.dtypes


fpath          object
subject        object
payload        object
contenttype    object
isSpam          int64
dtype: object

In [3]:
remove = ['\n', '\t', '\r','\\n', '\\t', '\\r', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '_', '+', '+', '{', '}', '[', ']', '\\', '|', ':', ';', '"', "'", ',', '.', '/', '?', '<', '>', '`', '~']
tmp = df['payload']
for r in remove:
    tmp = tmp.str.replace(r, '')
df['payload'] = tmp

C:\Users\research_laptop\AppData\Local\Temp\ipykernel_16336\3978132860.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tmp = tmp.str.replace(r, '')
C:\Users\research_laptop\AppData\Local\Temp\ipykernel_16336\3978132860.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tmp = tmp.str.replace(r, '')


In [4]:
df.head()

,fpath,subject,payload,contenttype,isSpam
0,C:\Users\research_laptop\Desktop\projects\scho...,Re: New Sequences Window,Date Wed 21 Aug 2002 105446 0500 ...,text/plain,0
1,C:\Users\research_laptop\Desktop\projects\scho...,[zzzzteana] RE: Alexander,Martin A postedTassos Papadopoulos the Greek s...,text/plain,0
2,C:\Users\research_laptop\Desktop\projects\scho...,[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow Thursday Aug...,text/plain,0
3,C:\Users\research_laptop\Desktop\projects\scho...,[IRR] Klez: The Virus That Won't Die,Klez The Virus That Wont Die Already the most ...,text/plain,0
4,C:\Users\research_laptop\Desktop\projects\scho...,Re: [zzzzteana] Nothing like mama used to make,in adding cream to spaghetti carbonara which...,text/plain,0


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(lowercase=True, stop_words='english')
cv_data = vect.fit_transform(df['payload'])
cv_data

<9353x190189 sparse matrix of type '<class 'numpy.int64'>'
	with 1240449 stored elements in Compressed Sparse Row format>

ifidf vectorizer drops accuracy of the model by 12%... could play with alpha but meh

In [6]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#vect = TfidfVectorizer(lowercase=True, stop_words='english')
#cv_data = vect.fit_transform(df['payload'])


In [7]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
#n_components = 200
#evr = [0]
#while sum(evr) < .98:
#    svd = TruncatedSVD(n_components, random_state=0)
#    count_vect_df = svd.fit_transform(cv_data)
#    evr = svd.explained_variance_ratio_
#    print(n_components, sum(evr))
#    n_components += 1


TruncatedSVD with MinMaxScaling produces poor results

In [8]:
#svd = TruncatedSVD(1000, random_state=0)
#count_vect_df = svd.fit_transform(cv_data)
#evr = svd.explained_variance_ratio_
#print(sum(evr))


In [9]:
#from sklearn.preprocessing import MinMaxScaler
#mms = MinMaxScaler()
#svd_scaled_data = mms.fit_transform(count_vect_df)
#count_vect_df = pd.DataFrame(svd_scaled_data, columns=svd.get_feature_names_out())
#combined_df = pd.concat([df, count_vect_df], axis=1)

In [10]:
count_vect_df = pd.DataFrame(cv_data.todense(), columns=vect.get_feature_names_out())
combined_df = pd.concat([df, count_vect_df], axis=1)

In [11]:
combined_df.drop(['fpath', 'subject'], axis=1, inplace=True)
combined_df.head()

,payload,contenttype,isSpam,00,000,0000,00000,000000,000000000000000000000000000032526amp3b223bbon203d2027273bwith2,000000000000000000000365000000000000000000000000000000000000000003640000,...,þàïáö¼ò,þë,þîñ,þõµ,þö,þööööà,þüg,ÿÿ,ÿÿã,ÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿÿò
0,Date Wed 21 Aug 2002 105446 0500 ...,text/plain,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Martin A postedTassos Papadopoulos the Greek s...,text/plain,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Man Threatens Explosion In Moscow Thursday Aug...,text/plain,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Klez The Virus That Wont Die Already the most ...,text/plain,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,in adding cream to spaghetti carbonara which...,text/plain,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [13]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans


In [28]:
sil = []
tmp = combined_df.drop(['isSpam', 'payload', 'contenttype'], axis=1)
kmeans = KMeans(n_clusters = 2).fit(tmp)
labels = kmeans.labels_
sil.append(silhouette_score(tmp, labels, metric = 'euclidean'))
print(sil)

[0.881927766585686]


who would have thought that 2 is the best nubmer of clusters to use...
this next bit was a huge waste of time.

In [15]:
#sil = []
#kmax = 50
#for k in range(2, kmax):
#    kmeans = KMeans(n_clusters=k).fit(tmp)
#    labels = kmeans.labels_
#    sil.append(silhouette_score(tmp,labels, metric='euclidean'))


In [16]:
#sil[sil.index(max(sil))]

In [17]:
#kmeans = KMeans(n_clusters=2).fit(tmp)
#labels = kmeans.labels_

In [18]:
label_df = pd.DataFrame({'labels':labels})
final_df = pd.concat([combined_df, label_df], axis=1)

In [19]:
tmp = final_df['contenttype']
cat = tmp.astype('category')
dummies = pd.get_dummies(cat)
final_df = pd.concat([final_df, dummies], axis='columns')
#final_df['ContentTypeCategory'] = tmp.iloc[:,[1]]
#final_df['ContentTypeCategory'] = tmp
#final_df.drop(['ContentTypeCategory'], axis=1, inplace=True)

In [20]:
final_df.drop(['payload', 'contenttype'], axis=1, inplace=True)

In [21]:
final_df['isSpam'].value_counts()

0    6954
1    2399
Name: isSpam, dtype: int64

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold, cross_val_score


In [23]:
import matplotlib.pyplot as plt
 
#filter rows of original data
filtered_label0 = final_df[labels == 0]
 
#plotting the results
#plt.scatter(filtered_label0[:,0] , filtered_label0[:,1])
#plt.show()

In [24]:
y = final_df['isSpam']
X = final_df.drop(['isSpam'], axis=1)
X


,00,000,0000,00000,000000,000000000000000000000000000032526amp3b223bbon203d2027273bwith2,000000000000000000000365000000000000000000000000000000000000000003640000,000000000000000003640000000000000000000000000000000032526amp3b223bbo,000000000020000000000000000000000000000000000365000000000000000000000000,00000000md,...,contenttype_multipart/related,contenttype_multipart/report,contenttype_multipart/signed,contenttype_text/html,contenttype_text/plain,contenttype_text/plain charset=us-ascii,contenttype_0,contenttype_1,contenttype_2,contenttype_3
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
9350,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
9351,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [25]:
y.isnull().values.any()
nans = final_df.loc[pd.isna(final_df["isSpam"]), :].index
for n in nans:
    print(df.iloc[n])

In [26]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

clf = MultinomialNB()
print(cross_val_score(clf, X, y, cv=kf, n_jobs=1))


[0.98610369 0.98610369 0.98824158 0.98342246 0.98128342]


In [27]:
accuracy = cross_val_score(clf, X, y, cv=kf, n_jobs=1)
print('Accuracy:', np.mean(accuracy), accuracy)

recall = cross_val_score(clf, X, y, cv=kf, scoring='recall', n_jobs=1)
print('Recall:', np.mean(recall), recall)

precision = cross_val_score(clf, X, y, cv=kf, scoring='precision', n_jobs=1)
print('Precision:', np.mean(precision), precision)

f1 = cross_val_score(clf, X, y, cv=kf, scoring='f1', n_jobs=1)
print('F1:', np.mean(f1), f1)

Accuracy: 0.9850309680259063 [0.98610369 0.98610369 0.98824158 0.98342246 0.98128342]
Recall: 0.9546089033608475 [0.96015936 0.96516393 0.96043956 0.94978166 0.9375    ]
Precision: 0.9866594295710195 [0.98770492 0.98125    0.99092971 0.98194131 0.99147122]
F1: 0.9703297305616886 [0.97373737 0.9731405  0.97544643 0.96559378 0.96373057]



make roc curve for naive bayes
make confusion matrix
most common words grouped by category 